In [1]:
import numpy
trainX = numpy.load('tinyX.npy') # this should have shape (26344, 3, 64, 64)
trainY = numpy.load('tinyY.npy') 
testX = numpy.load('tinyX_test.npy') # (6600, 3, 64, 64)

# to visualize only
#import scipy.misc
#scipy.misc.imshow(trainX[0].transpose(2,1,0)) # put RGB channels last

from matplotlib import pyplot as plt
#plt.imshow(trainX[0].transpose(2,1,0))
#plt.show()

import tensorflow
import tflearn
from tflearn.data_utils import shuffle, to_categorical
import sklearn
from sklearn.cross_validation import train_test_split
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from __future__ import division, print_function, absolute_import

In [2]:
#Dataloading and preprocessing
trainX, trainY = shuffle(trainX, trainY) #what is this shuffling exactly doing?

X_train, X_valid, y_train, y_valid = train_test_split(trainX, trainY, test_size=0.80, random_state=42)
y_train = to_categorical(y_train, 40)
y_valid = to_categorical(y_valid, 40)

trainY

array([0, 2, 0, ..., 0, 0, 4], dtype=uint8)

In [ ]:
#Image processing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center() #what?
img_prep.add_featurewise_stdnorm() #what?

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [ ]:
#Convolutional Neural Network

#creating data model
cnn_network = input_data(shape=[None, 64, 64, 3], data_preprocessing=img_prep, data_augmentation=img_aug)

#creating 2D Convolution
cnn_network = conv_2d(cnn_network, 64, 3, activation='relu')

cnn_network = max_pool_2d(cnn_network, 2)

cnn_network = conv_2d(cnn_network, 128, 3, activation='relu')
cnn_network = conv_2d(cnn_network, 128, 3, activation='relu')

cnn_network = max_pool_2d(cnn_network, 2)

cnn_network = fully_connected(cnn_network, 512, activation='relu')

cnn_network = dropout(cnn_network, 0.5)

cnn_network = fully_connected(cnn_network, 40, activation='softmax')

cnn_network = regression(cnn_network, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

#Train the classifier now
model = tflearn.DNN(cnn_network, tensorboard_verbose=0)
model.fit(X_train, y_train, n_epoch=50, shuffle=True, validation_set=(X_valid, y_valid), show_metric=True, batch_size=96)


